In [1]:
# !pip install --upgrade mxnet
# !pip install autogluon

zsh:1: command not found: pip
zsh:1: command not found: pip


In [4]:
# import autogluon.core as ag
# from autogluon.tabular import TabularPrediction as task

ImportError: cannot import name 'TabularPrediction' from 'autogluon.tabular' (/opt/anaconda3/lib/python3.8/site-packages/autogluon/tabular/__init__.py)

In [5]:
import autogluon.core as ag
from autogluon.tabular import TabularPrediction as task
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import gc
import math
import os
import glob

ImportError: cannot import name 'TabularPrediction' from 'autogluon.tabular' (/opt/anaconda3/lib/python3.8/site-packages/autogluon/tabular/__init__.py)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
os.getcwd()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'os' is not defined

In [ ]:
df = pd.read_csv("Train.csv")
small_ids = np.array(pd.read_csv("Small_Ids.csv")) #hopefully this works lol
df.head()

,Unique,Id,Day,Lat,Lon,Elevation,Emissions,Forest,Roads,Streets,Plumes_High,Plumes_Med,Plumes_Low,Max_Temp,Max_Wind,Precip,Rel_Humidity,Wind_Dir,BLH,AOD,PM
0,1131_1,1131,1,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,53.4,5.3,0.0,67.0,251.0,758.111816,NaN,3.7
1,1131_100,1131,100,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,64.1,3.6,0.0,48.0,308.0,1155.824219,NaN,2.6
2,1131_101,1131,101,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,62.7,5.1,0.0,64.0,200.0,1162.292725,NaN,2.9
3,1131_102,1131,102,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,67.7,7.9,0.0,68.0,221.0,263.273407,NaN,0.5
4,1131_104,1131,104,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,60.1,6.5,0.0,44.0,314.0,1268.919434,0.097667,2.4


In [ ]:
def bias(true, pred):
    return (pred-true).mean()

def r2(true, pred):
    numerator = ((pred - pred.mean())*(true - true.mean())).sum()
    sd1 = math.sqrt(sum(np.square(pred - pred.mean())))
    sd2 = math.sqrt(sum(np.square(true - true.mean())))
    return np.square(numerator/(sd1*sd2))

def nse(true, pred):
    return 1 - sum(np.square(pred - true))/sum(np.square(true - true.mean()))

def mae(true, pred):
    return ((pred - true).abs()).mean()

def mse(true, pred):
    return np.square(pred - true).mean()

def rmse(true, pred):
    return math.sqrt(np.square(pred - true).mean())

In [ ]:
def cv(hyperparameters):
    n_fold = len(set(df['Id']))
    kf = GroupKFold(n_fold)
    split = kf.split(df, groups = df['Id'])
    
    metric_df = pd.DataFrame(columns = ['station', 'MAE', 'MSE', 'RMSE', 'NSE', 'Bias', 'R2']) #this is for plotting errors
    
    for i, (train_idx, test_idx) in enumerate(split):
        print(f'Starting training fold {i}.')
        _ = gc.collect()

        features = ['Day', 'Lat', 'Lon', 'Elevation', 'Emissions', 'Forest',
            'Roads', 'Streets', 'Plumes_High', 'Plumes_Med', 'Plumes_Low',
            'Max_Temp', 'Max_Wind', 'Precip', 'Rel_Humidity', 'Wind_Dir', 'BLH',
            'AOD']
        label_column = 'PM'

        X = df[features]
        y = df[label_column]

        train_data = task.Dataset(pd.concat([X.loc[train_idx], y.loc[train_idx]], axis=1))
        test_data = task.Dataset(pd.concat([X.loc[test_idx], y.loc[test_idx]], axis=1))

        predictor = task.fit(train_data=train_data, 
                             label=label_column, 
                             hyperparameters=hyperparameters, 
                             # time_limits = 60*3,
                             # hyperparameter_tune=True, 
                             eval_metric='r2')

        y_test = test_data[label_column]
        # print("Actual: ", y_test)
        test_data_nolab = test_data.drop(labels = [label_column], axis = 1)

        y_pred = predictor.predict(test_data_nolab)
        # print("Predictions: ", y_pred)

        # perf = predictor.evaluate_predictions(y_true = y_test, y_pred = y_pred, auxiliary_metrics = True)
        station = df.loc[test_idx]['Id'].iloc[0]
        MAE = mae(y_test, y_pred)
        MSE = mse(y_test, y_pred)
        RMSE = rmse(y_test, y_pred)
        NSE = nse(y_test, y_pred)
        BIAS = bias(y_test, y_pred)
        R2 = r2(y_test, y_pred)

        metric_df = metric_df.append({'station': station, 'MAE': MAE, 'MSE': MSE , 'RMSE': RMSE, 'NSE': NSE, 'Bias': BIAS, 'R2':R2}, ignore_index = True)

    return metric_df

In [ ]:
# The defaults: 
# 'NN': [{}]
# 'GBM': [{}]
# 'CAT': [{}]
# 'RF': [{'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}]
# 'XT': [{'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}]
# 'KNN': [{'weights': 'uniform', 'AG_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'AG_args': {'name_suffix': 'Dist'}}]
# 'custom': [{'num_boost_round': 10000, 'num_threads': -1, 'objective': 'regression', 'verbose': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'two_round': True, 'seed_value': 0, 'AG_args': {'model_type': 'GBM', 'name_suffix': 'Custom', 'disable_in_hpo': True}}]}}

CAT = cv({'CAT': {}})
CAT.to_csv("/content/drive/My Drive/NASA PM_estimation/CAT.csv", index = False)

In [ ]:
NN = cv({'NN': {}})
NN.to_csv("/content/drive/My Drive/NASA PM_estimation/NN.csv", index = False)

In [ ]:
GBM = cv({'GBM': {}})
GBM.to_csv("/content/drive/My Drive/NASA PM_estimation/GBM.csv", index = False)

In [ ]:
RF = cv({'RF': {'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}})
RF.to_csv("/content/drive/My Drive/NASA PM_estimation/RF.csv", index = False)

In [ ]:
XT = cv({'XT': [{'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}]})
XT.to_csv("/content/drive/My Drive/NASA PM_estimation/XT.csv", index = False)

In [ ]:
KNNunif = cv({'KNN': [{'weights': 'uniform', 'AG_args': {'name_suffix': 'Unif'}}]})
KNNunif.to_csv("/content/drive/My Drive/NASA PM_estimation/KNNunif.csv", index = False)

In [ ]:
KNNdist = cv({'KNN': [{'weights': 'distance', 'AG_args': {'name_suffix': 'Dist'}}]})
KNNdist.to_csv("/content/drive/My Drive/NASA PM_estimation/KNNdist.csv", index = False)

In [ ]:
GBMcustom = cv({'custom': [{'num_boost_round': 10000, 'num_threads': -1, 'objective': 'regression', 'verbose': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'two_round': True, 'seed_value': 0, 'AG_args': {'model_type': 'GBM', 'name_suffix': 'Custom', 'disable_in_hpo': True}}]})
GBMcustom.to_csv("/content/drive/My Drive/NASA PM_estimation/GMBcustom.csv", index = False)

In [ ]:
Averages = pd.DataFrame(columns = ['Model', 'MAE', 'MSE', 'RMSE', 'NSE', 'Bias', 'R2'])

paths = glob.glob("/content/drive/My Drive/NASA PM_estimation/*.csv")

for path in paths:
    f = os.path.split(path)[1]
    Model = os.path.splitext(f)[0]
    Table = pd.read_csv(path)
    MAE = Table['MAE'].mean()
    MSE = Table['MSE'].mean()
    RMSE = Table['RMSE'].mean()
    NSE = Table['NSE'].mean()
    BIAS = Table['Bias'].mean()
    R2 = Table['R2'].mean()
    Averages = Averages.append({'Model': Model, 'MAE': MAE, 'MSE': MSE , 'RMSE': RMSE, 'NSE': NSE, 'Bias': BIAS, 'R2':R2}, ignore_index = True)

Averages.to_csv("/content/drive/My Drive/NASA PM_estimation/means/ModelMeans.csv", index = False)

In [ ]:
small_ids = pd.read_csv("Small_Ids.csv")['x']
small_ids = [float(i) for i in small_ids]

Averages = pd.DataFrame(columns = ['Model', 'MAE', 'MSE', 'RMSE', 'NSE', 'Bias', 'R2'])

paths = glob.glob("/content/drive/My Drive/NASA PM_estimation/*.csv")

for path in paths:
    f = os.path.split(path)[1]
    Model = os.path.splitext(f)[0]
    print(Model)
    Table = pd.read_csv(path)
    ##FILTER SO ONLY SMALL
    # print(Table['station'])
    # print(small_ids['x'])
    Table = Table[Table['station'].isin(small_ids)]
    MAE = Table['MAE'].mean()
    MSE = Table['MSE'].mean()
    RMSE = Table['RMSE'].mean()
    NSE = Table['NSE'].mean()
    BIAS = Table['Bias'].mean()
    R2 = Table['R2'].mean()
    Averages = Averages.append({'Model': Model, 'MAE': MAE, 'MSE': MSE , 'RMSE': RMSE, 'NSE': NSE, 'Bias': BIAS, 'R2':R2}, ignore_index = True)

Averages.to_csv("/content/drive/My Drive/NASA PM_estimation/means/SmallGridModelMeans.csv", index = False)

CAT
GBM
NN
RF
NN_tune
KNNunif
KNNdist
GMBcustom
XT


In [ ]:
small_grid = pd.read_csv("Small_Grid.csv")

In [ ]:
small_grid[small_grid['Day'].isna()]

,Unique,Id,Day,Lat,Lon,Elevation,Emissions,Forest,Roads,Streets,Plumes_High,Plumes_Med,Plumes_Low,Max_Temp,Max_Wind,Precip,Rel_Humidity,Wind_Dir,BLH,AOD,PM


In [ ]:
small_grid['Day']

0          1
1         10
2        100
3        101
4        102
        ... 
68708    133
68709    134
68710    135
68711    136
68712    137
Name: Day, Length: 68713, dtype: int64

In [ ]:
def fitgrid(hyperparameters):
  features = ['Day', 'Elevation', 'Emissions', 'Forest',
      'Roads', 'Streets', 'Plumes_High', 'Plumes_Med', 'Plumes_Low',
      'Max_Temp', 'Max_Wind', 'Precip', 'Rel_Humidity', 'Wind_Dir', 'BLH',
      'AOD', 'PM']
  label_column = 'PM'

  train_data = task.Dataset(df[features])
  test_data = task.Dataset(small_grid[features])

  predictor = task.fit(train_data=train_data, 
                        label=label_column, 
                        hyperparameters=hyperparameters, 
                        # time_limits = 60*3,
                        # hyperparameter_tune=True, 
                        eval_metric='r2')

  test_data_nolab = test_data.drop(labels = [label_column], axis = 1)
  # y_pred = predictor.predict(test_data_nolab)
  test_data['y_pred'] = predictor.predict(test_data_nolab)
  print(test_data.head())
  test_data['Id'] = small_grid['Id']
  test_data['Day'] = test_data['Day'].apply(lambda x: 'Day'+str(x))
  test_data = test_data.pivot(index='Id', columns='Day', values='y_pred')

  return test_data

In [ ]:
GBMcustom = fitgrid({'custom': [{'num_boost_round': 10000, 'num_threads': -1, 'objective': 'regression', 'verbose': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'two_round': True, 'seed_value': 0, 'AG_args': {'model_type': 'GBM', 'name_suffix': 'Custom', 'disable_in_hpo': True}}]})
GBMcustom.to_csv("/content/drive/My Drive/NASA PM_estimation/grids/GMBcustom.csv", index = True)

In [ ]:
GBMcustom.mean(axis = 1).to_csv("/content/drive/My Drive/NASA PM_estimation/grids/GMBcustom_Avg.csv", index = True)

In [ ]:
CAT = fitgrid({'CAT': {}})
CAT.to_csv("/content/drive/My Drive/NASA PM_estimation/grids/CAT_Grid.csv", index = True)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200808_022131/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200808_022131/
AutoGluon Version:  0.0.12
Train Data Rows:    10082
Train Data Columns: 17
Preprocessing data ...
Here are the first 10 unique label values in your data:  [3.7, 2.6, 2.9, 0.5, 2.4, 1.8, 1.2, 3.4, 2.1, 5.4]
AutoGluon infers your prediction problem is: regression  (because dtype of label-column == float and many unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 10082 data points with 16 features
Original Features (raw dtypes):
	int64 features: 5
	float64 features: 11
Original Features (inferred dtypes):
	int features: 5
	float features: 11
Generated Features (special dtypes):
Final Features (raw dtypes):
	int features: 5
	float features:

ValueError: ignored

In [ ]:
CAT.mean(axis = 1).to_csv("/content/drive/My Drive/NASA PM_estimation/grids/CAT_Avg.csv", index = True)

In [ ]:
GBMcustom = pd.read_csv("/content/drive/My Drive/NASA PM_estimation/grids/GMBcustom.csv", index_col = 0)
#GBMcustom.median(axis = 1).to_csv("/content/drive/My Drive/NASA PM_estimation/grids/GMBcustom_Med.csv", index = True)
GBMcustom[['Day244', 'Day245', 'Day282', 'Day283', 'Day284', 'Day285', 'Day286']].to_csv("/content/drive/My Drive/NASA PM_estimation/grids/GBMc_DOI.csv", index = True)

In [ ]:
CAT = pd.read_csv("/content/drive/My Drive/NASA PM_estimation/grids/CAT.csv", index_col = 0)
# CAT.median(axis = 1).to_csv("/content/drive/My Drive/NASA PM_estimation/grids/CAT_med.csv", index = True)
CAT[['Day244', 'Day245', 'Day282', 'Day283', 'Day284', 'Day285', 'Day286']].to_csv("/content/drive/My Drive/NASA PM_estimation/grids/CAT_DOI.csv", index = True)